# 0.0 Imports 

In [12]:
from prometheus_api_client import PrometheusConnect
from datetime import datetime
import pandas as pd
import os
import jornadas
import numpy as np

# 1.0 Extrator

In [2]:
# usa a query istio
def mean_requests_max_istio(app, data_inicio, data_fim):
    
    # criando dataframe com a quantidade de reqs/s, data e posição na tabela
    request_count_istio = pd.DataFrame()
    for posicao, numero in enumerate(list(prom.custom_query_range(query='sum(rate(istio_requests_total{app="' + app + '"}[1m]))', start_time=datetime.strptime(data_inicio, '%Y/%m/%d %H:%M'), end_time=datetime.strptime(data_fim, '%Y/%m/%d %H:%M'), step='1m'))[0]['values']):
        
        linha = pd.DataFrame({
            'posicao': [posicao],
            'data': numero[0],
            'valor': (float(numero[1]) * 60)
        })

        request_count_istio = pd.concat([request_count_istio, linha])


    # identificando pico de requests/s dentro do período de tempo passado e tirando uma média de reqs/s dos últimos 10 min a partir do pico 
    valores_pico = pd.DataFrame()
    for n in range(len(request_count_istio)):
        if (request_count_istio['posicao'].iloc[n] >= int(request_count_istio[request_count_istio['valor'] == max(request_count_istio['valor'])]['posicao'].iloc[0]) - 10) & (request_count_istio['posicao'].iloc[n] <= int(request_count_istio[request_count_istio['valor'] == max(request_count_istio['valor'])]['posicao'].iloc[0])):
            
            linha1 = pd.DataFrame({
                'valor': [float(request_count_istio['valor'].iloc[n])]
            }) 

            valores_pico = pd.concat([valores_pico, linha1])

    return round(valores_pico['valor'].mean()) 
    #return request_count_istio['valor'].mean()

In [11]:
# usa a query nginx
def mean_requests_max_nginx(app, data_inicio, data_fim):
    
    # criando dataframe com a quantidade de reqs/s, data e posição na tabela
    request_count_nginx = pd.DataFrame()
    for posicao, numero in enumerate(list(prom.custom_query_range(query='sum(rate(nginx_ingress_controller_requests{service="' + app + '"}[1m]))', start_time=datetime.strptime(data_inicio, '%Y/%m/%d %H:%M'), end_time=datetime.strptime(data_fim, '%Y/%m/%d %H:%M'), step='1m'))[0]['values']):
        
        linha = pd.DataFrame({
            'posicao': [posicao],
            'data': numero[0],
            'valor': (float(numero[1]) * 60)
        })

        request_count_nginx = pd.concat([request_count_nginx, linha])


    # identificando pico de requests/s dentro do período de tempo passado e tirando uma média de reqs/s dos últimos 10 min a partir do pico 
    valores_pico = pd.DataFrame()
    for n in range(len(request_count_nginx)):
        if (request_count_nginx['posicao'].iloc[n] >= int(request_count_nginx[request_count_nginx['valor'] == max(request_count_nginx['valor'])]['posicao'].iloc[0]) - 10) & (request_count_nginx['posicao'].iloc[n] <= int(request_count_nginx[request_count_nginx['valor'] == max(request_count_nginx['valor'])]['posicao'].iloc[0])):
            
            linha1 = pd.DataFrame({
                'valor': [float(request_count_nginx['valor'].iloc[n])]
            }) 

            valores_pico = pd.concat([valores_pico, linha1])

    return round(valores_pico['valor'].mean()) 

In [13]:
# transformando listas de dicts em dataframes
df = pd.concat([pd.DataFrame(data=jornadas.compra), pd.DataFrame(data=jornadas.b2b), pd.DataFrame(data=jornadas.figital)]).reset_index()

# criando uma lista com os nomes dos servidores 
server = list(df['server'].copy())

# limpando a coluna "index" e a coluna "server"
df = df.drop(['index', 'server'], axis=1)

df['bateria'] = np.nan

## 1.1 Extração

In [14]:
# colhendo informações a respeito do teste
qtd_baterias = input('Quantidade de bateria no teste')

metrica =''
while True:
    if 'nginx' not in metrica or 'istio' not in metrica:
        # escolhendo qual métrica usar
        metrica = input('Qual métrica usar para extrair os dados?(Métricas aceitas: nginx/istio)')
        if 'nginx' not in metrica or 'istio' not in metrica:
            print('Métrica não localizada. As métricas aceitas são: nginx ou istio')
    
    if 'nginx' in metrica or 'istio' in metrica:
        break

# colhendo horários de início e término das baterias
if int(qtd_baterias) >= int(1):
    data_inicio = input('Data Hora de Início da Bateria 1')
    data_fim = input('Data Hora de Término da Bateria 1')

if int(qtd_baterias) >= int(2):
    data_inicio2 = input('Data Hora de Início da Bateria 2')
    data_fim2 = input('Data Hora de Término da Bateria 2') 

if int(qtd_baterias) >= int(3):
    data_inicio3 = input('Data Hora de Início da Bateria 3')
    data_fim3 = input('Data Hora de Término da Bateria 3')


# loop para preencher os dados em cada uma das baterias do teste 
for i in range(int(qtd_baterias)):

    # alimentando a variável com horário da bateria que será extraída na sequência 
    if int(i+1) == 2:
        data_inicio = data_inicio2
        data_fim = data_fim2

    if int(i+1) == 3:
        data_inicio = data_inicio3
        data_fim = data_fim3

    # loop para realizar a extração por bateria
    if 'istio' in metrica:
        for numero, coluna in enumerate(df):
            if numero > 2:
                for num, app in enumerate(df[coluna]):
                    prom = PrometheusConnect(url =f"{server[num]}", disable_ssl=True)
                    try:
                        df[coluna].iloc[num] = mean_requests_max_istio(str(app), data_inicio, data_fim)
                    except:
                        df[coluna].iloc[num] = np.nan
    
    if 'nginx' in metrica:
        for numero, coluna in enumerate(df):
            if numero > 2:
                for num, app in enumerate(df[coluna]):
                    prom = PrometheusConnect(url =f"{server[num]}", disable_ssl=True)
                    try:
                        df[coluna].iloc[num] = mean_requests_max_nginx(str(app), data_inicio, data_fim)
                    except:
                        df[coluna].iloc[num] = np.nan

     

    # salvando a cópia da base principal em uma base apartada 
    if int(i+1) == 1:
        df['bateria'] = 1
        bateria1 = df.copy()
    
    if int(i+1) == 2:
        df['bateria'] = 2
        bateria2 = df.copy()
    
    if int(i+1) == 3:
        df['bateria'] = 3
        bateria3 = df.copy()
    
    # transformando listas de dicts em dataframes
    df = pd.concat([pd.DataFrame(data=jornadas.compra), pd.DataFrame(data=jornadas.b2b), pd.DataFrame(data=jornadas.figital)]).reset_index()

    # criando uma lista com os nomes dos servidores 
    server = list(df['server'].copy())

    # limpando a coluna "index" e a coluna "server"
    df = df.drop(['index', 'server'], axis=1)

    df['bateria'] = np.nan

# unindo e salvando as baterias em uma base consolidada
if int(qtd_baterias) == 1:
    df = bateria1.copy()

if int(qtd_baterias) == 2:
    df = pd.concat([bateria1, bateria2])

if int(qtd_baterias) == 3:
    df = pd.concat([bateria1, bateria2, bateria3])

Métrica não localizada. As métricas aceitas são: nginx ou istio


C:\Users\4960021521\AppData\Local\Temp\ipykernel_28792\347026907.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[coluna].iloc[num] = mean_requests_max_nginx(str(app), data_inicio, data_fim)
C:\Users\4960021521\AppData\Local\Temp\ipykernel_28792\347026907.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[coluna].iloc[num] = mean_requests_max_nginx(str(app), data_inicio, data_fim)
C:\Users\4960021521\AppData\Local\Temp\ipykernel_28792\347026907.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pa

## 1.2 Resultado

In [6]:
df #<- data/hora do dado 2023/04/20 10:00hs/11:00hs istio_requests_total

,jornada,tribo,produto,reqs_cb,reqs_ex,reqs_pf,reqs_corp,bateria
0,Compra,vitrine,carrinho,32,5,8,NaN,1
1,Compra,carrinho-front,apollo,26,5,6,NaN,1
2,Compra,check,checkout,22,3,5,NaN,1
3,b2b,parcerias,checkout-front,12,2,2,NaN,1
4,b2b,parcerias,kong_proxy,NaN,NaN,NaN,0,1
5,b2b,parcerias,kong,NaN,NaN,NaN,NaN,1
6,teste,teste,teste,12,2,2,NaN,1
7,teste,teste,teste,NaN,NaN,NaN,NaN,1
8,teste,teste,teste,NaN,NaN,NaN,NaN,1


In [10]:
df #<- data/hora do dado 2023/04/20 10:00hs/11:00hs nginx_ingress_controller_requests

,jornada,tribo,produto,reqs_cb,reqs_ex,reqs_pf,reqs_corp,bateria
0,Compra,vitrine,carrinho,1061,170,263,NaN,1
1,Compra,carrinho-front,apollo,695,199,174,NaN,1
2,Compra,check,checkout,1027,135,217,NaN,1
3,b2b,parcerias,checkout-front,407,61,49,NaN,1
4,b2b,parcerias,kong_proxy,6062,839,971,NaN,1
5,b2b,parcerias,kong,NaN,NaN,NaN,836,1
6,teste,teste,teste,407,61,49,NaN,1
7,teste,teste,teste,NaN,NaN,NaN,NaN,1
8,teste,teste,teste,NaN,NaN,NaN,836,1
